In [1]:
import random as rd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# Queue of Events
class EventsQueue:
    def __init__(self):
        self.globalTime = 0
        self.MEvents = []

    def QueueSize(self):
        return len(self.MEvents)

    def add_event(self, MEvent):
        count = len(self.MEvents)
        if count == 0 or MEvent.eTime >= self.MEvents[count - 1].eTime:
            self.MEvents.append(MEvent)
            return 0

        for i in range(0, count-1):
            if MEvent.eTime >= self.MEvents[i].eTime:
                if MEvent.eTime < self.MEvents[i + 1].eTime:
                    self.MEvents.insert(i + 1, MEvent)
                    return 0

    def process_next_event(self):
        if len(self.MEvents) == 0:
            return 0
        self.MEvents[0].Execute()
        self.globalTime = self.MEvents[0].eTime
        del self.MEvents[0]

In [13]:
# Discrete Event System Specification
# Добавил в пример для 4-ой лабораторной множество серверов и множество очередей
class DEVS:
    def __init__(self, task, servers_num, queues_num):
        self.servers_num = servers_num
        self.queues_num = queues_num
        self.EQ = EventsQueue()
        self.GlobalTime = 0.0
        self.serversIdle = {i: True for i in range(servers_num)}
        self.customer_queues = {i: [] for i in range(queues_num)}

        # simulation attributes
        self.customerQueue = []
        self.stats = []
        self.newId = 0
        self.serverIdle = True
        self.lastServedTime = 0  # for Idle time
        self.lastServedTimes = {i: 0 for i in range(servers_num)}  # for Idle time

    def process_next_event(self):
        self.EQ.process_next_event()
        self.GlobalTime = self.EQ.globalTime

    def check_queues(self):
        free_servers = [i for i in range(self.servers_num) if self.serversIdle[i]]
        servers_queues = {i: len(self.customer_queues[i]) for i in range(self.queues_num)}
        return free_servers, servers_queues


In [3]:
def print_statistics(devs_list):

    # 1) Average waiting time in queue
    avTimeInQueue = [sum([x.waitingTimeInQueue for x in devs.stats]) / len(devs.stats) for devs in devs_list]
    print("\nAverage waiting times: " + ' '.join(['%.2f' % (i,) for i in avTimeInQueue]))

    # 2) Probability that a customer has to wait
    probToWait = [len([x for x in devs.stats if x.waitingTimeInQueue > 0]) / len(devs.stats) for devs in devs_list]
    print("\nProbabilities that a customer has to wait: " + ' '.join(['%.2f' % (i,) for i in probToWait]))

    # 3) Probability of an Idle server
    # probIdle = sum([x.idleTimeOfServer for x in devs.stats]) / devs.GlobalTime
    probIdle = [1 - np.prod([1 - sum([x.idleTimeOfServer for x in devs.stats if x.server_ind == i]) / devs.GlobalTime
                                      for i in range(len(devs.serversIdle))]) for devs in devs_list]
    print("\nProbabilities of an Idle server: " + ' '.join(['%.2f' % (i,) for i in probIdle]))

    # 4) Average service time (theoretical 3.2)
    avServiceTime = [sum([x.serviceTime for x in devs.stats]) / len(devs.stats) for devs in devs_list]
    print("\nAverage service times: " + ' '.join(['%.2f' % (i,) for i in avServiceTime]))

    # 5) Average time between arrivals (theoretical 4.5)
    avTimeBetwArr = [sum([x.interArrivalTime for x in devs.stats]) / (len(devs.stats) - 1) for devs in devs_list]
    print("\nAverage times between arrivals: " + ' '.join(['%.2f' % (i,) for i in avTimeBetwArr]))

    # 6) Average waiting time for those who wait
    numOfCustWhoWait = [len([x for x in devs.stats if x.waitingTimeInQueue > 0]) for devs in devs_list]
    avTimeWhoWait = [sum([x.waitingTimeInQueue for x in devs.stats]) / numOfCustWhoWait[i] if numOfCustWhoWait[i] else 0 
                     for i, devs in enumerate(devs_list)]
    print("\nAverage waiting times for those who wait: " + ' '.join(['%.2f' % (i,) for i in avTimeWhoWait]))

    # 7) Average time a customer spends in the system
    avTimeInTheSystem1 = [sum([x.timeInSystem for x in devs.stats]) / len(devs.stats) for devs in devs_list]
    print("\nAverage times a customer spends in the system: " + ' '.join(['%.2f' % (i,) for i in avTimeInTheSystem1]))

    # avTimeInTheSystem2 = avTimeInQueue + avServiceTime
    # print("\nAverage time a customer spends in the system (alternative): {0:.2f}".format(avTimeInTheSystem2))

In [14]:
# ---- Customer Statistics ----
class CustomerStat:
    def __init__(self):
        self.id = -1
        self.arrivalTime = -1
        self.serviceTime = -1
        self.interArrivalTime = 0
        self.serviceBegins = -1
        self.waitingTimeInQueue = 0
        self.serviceEnds = -1
        self.timeInSystem = -1
        self.idleTimeOfServer = 0
        self.server_ind = -1

In [15]:
# ---- Arrival Event ----
class ArrivalEvent:
    def __init__(self, devs, task, maxAngents, arrivalRateMin, 
                 arrivalRateMax, custm, verbose, threshold=None):
        self.devs = devs
        self.task = task
        self.maxAngents = maxAngents
        self.arrivalRateMin = arrivalRateMin
        self.arrivalRateMax = arrivalRateMax
        self.custm = custm
        self.eTime = 0.0
        if task == 1:
            self.ar_server = rd.randint(0, devs.servers_num - 1)
        self.threshold = threshold
        self.verbose = verbose

    def Execute(self):
        customer = CustomerStat()
        customer.id = self.devs.newId
        customer.arrivalTime = self.eTime
        if len(self.devs.stats) > 0:
            customer.interArrivalTime = customer.arrivalTime - self.devs.stats[-1].arrivalTime

        if self.verbose:
            print("Time %d" % self.eTime, " Arrival Event of agent {0}".format(customer.id))
            print(self.devs.EQ.MEvents)
        if self.devs.newId < self.maxAngents - 1:
            next_arrival = ArrivalEvent(self.devs, self.task, self.maxAngents, self.arrivalRateMin, self.arrivalRateMax,
                                        self.custm, self.verbose, self.threshold)
            next_arrival.eTime = self.eTime + rd.randint(self.arrivalRateMin, self.arrivalRateMax)
            self.devs.EQ.add_event(next_arrival)

        def add_for_service(server_ind, queue_ind):
            if self.verbose:
                print("server is Busy")
            Service = ServiceEvent(self.devs, self.task, self.custm, server_ind, queue_ind, self.verbose)
            serviceTime = self.custm.rvs()
            customer.serviceTime = serviceTime
            customer.serviceBegins = self.eTime  # current time
            Service.eTime = self.eTime + serviceTime
            Service.id = customer.id
            self.devs.EQ.add_event(Service)

        if self.task == 1:
            if self.devs.serversIdle[self.ar_server]:
                self.devs.serversIdle[self.ar_server] = False
                add_for_service(self.ar_server, self.ar_server)
            else:
                self.devs.customer_queues[self.ar_server].append(customer.id)
                if self.verbose:
                    print("customerQueue = %d" % len(self.devs.customer_queues[self.ar_server]))
        else:
            free_servers, servers_queues = self.devs.check_queues()
            if len(free_servers) > 0:
                server_ind = rd.choice(free_servers)
                queue_ind = server_ind if self.task == 2 else 0
                self.devs.serversIdle[server_ind] = False
                add_for_service(server_ind, queue_ind)
            else:
                queue_ind = min(servers_queues, key=servers_queues.get) if self.task == 2 else 0
                self.devs.customer_queues[queue_ind].append(customer.id)
                if self.verbose:
                    print("customerQueue = %d" % len(self.devs.customer_queues[queue_ind]))

                if self.task == 4:
                    if max(servers_queues.values()) >= self.threshold:
                        self.devs.serversIdle[self.devs.servers_num] = True
                        if not self.devs.lastServedTimes.get(self.devs.servers_num):
                            self.devs.lastServedTimes[self.devs.servers_num] = 0
                        self.devs.servers_num += 1
                        if self.verbose:
                            print("Number of servers = %d" % self.devs.servers_num)

        self.devs.newId = self.devs.newId + 1
        self.devs.stats.append(customer)

In [ ]:
# ---- Service (END) Event ----
class ServiceEvent:
    def __init__(self, devs, task, custm, server_ind, queue_ind, verbose):
        self.devs = devs
        self.task = task
        self.custm = custm
        self.server_ind = server_ind
        self.queue_ind = queue_ind
        self.eTime = 0.0
        self.id = 0
        self.verbose = verbose

    def Execute(self):
        ind = [i for i, val in enumerate(self.devs.stats) if val.id == self.id][0]
        self.devs.stats[ind].serviceEnds = self.eTime
        self.devs.stats[ind].timeInSystem = self.devs.stats[ind].serviceEnds - self.devs.stats[ind].arrivalTime
        self.devs.stats[ind].waitingTimeInQueue = self.devs.stats[ind].serviceBegins - self.devs.stats[ind].arrivalTime # 0 without queue
        self.devs.stats[ind].idleTimeOfServer = self.devs.stats[ind].serviceBegins - self.devs.lastServedTimes[self.server_ind]
        self.devs.stats[ind].server_ind = self.server_ind

        if self.verbose:
            print("Time %d" % self.eTime, "Service finished")

        def add_for_service(qid, server_ind, queue_ind):
            qind = [i for i, val in enumerate(self.devs.stats) if val.id == qid][0]
            Service = ServiceEvent(self.devs, self.task, self.custm, server_ind, queue_ind, self.verbose)
            serviceTime = self.custm.rvs()
            Service.eTime = self.eTime + serviceTime
            Service.id = qid
            self.devs.stats[qind].serviceBegins = self.eTime
            self.devs.stats[qind].serviceTime = serviceTime
            self.devs.EQ.add_event(Service)
            if self.verbose:
                print("take new customer from the queue")

        if len(self.devs.customer_queues[self.queue_ind]) > 0 and self.server_ind < self.devs.servers_num:
            qid = self.devs.customer_queues[self.queue_ind].pop(0)
            add_for_service(qid, self.server_ind, self.queue_ind)
        else:
            self.devs.serversIdle[self.server_ind] = True
            if self.verbose:
                print("server is Idle (do nothing)")

        self.devs.lastServedTimes[self.server_ind] = self.eTime


def run_devs(task, servers_num, maxAngents, arrivalRateMin, arrivalRateMax, custm, threshold, verbose=False):
    # run simulation
    task = task
    queues_num = servers_num if task == 1 or task == 2 else 1
    devs = DEVS(task, servers_num, queues_num)
    AE = ArrivalEvent(devs, task, maxAngents, arrivalRateMin, arrivalRateMax, custm, verbose, threshold)
    devs.EQ.add_event(AE)

    # --- SIMULATION ---
    while devs.EQ.QueueSize() > 0:
        devs.process_next_event()

    return devs

In [4]:
# Параметры системы
maxAngents = 100
arrivalRateMin = 1
arrivalRateMax = 4
service_xk = np.arange(6) + 5
service_pk = (0.1, 0.2, 0.3, 0.25, 0.1, 0.05)
custm = stats.rv_discrete(name='custm', values=(service_xk, service_pk))
threshold = 4

In [5]:
servers_num = 3
devs = []
for task in range(1, 5):
    devs.append(run_devs(task, servers_num, maxAngents, arrivalRateMin, arrivalRateMax, custm, threshold))

In [6]:
print_statistics(devs)


Average waiting times: 11.85 4.96 0.84 1.72

Probabilities that a customer has to wait: 0.72 0.62 0.41 0.50

Probabilities of an Idle server: 0.43 0.23 0.28 0.24

Average service times: 7.12 7.41 7.38 7.20

Average times between arrivals: 2.71 2.60 2.74 2.53

Average waiting times for those who wait: 16.46 8.00 2.05 3.44

Average times a customer spends in the system: 18.97 12.37 8.22 8.92


In [7]:
servers_num = 4
devs = []
for task in range(1, 5):
    devs.append(run_devs(task, servers_num, maxAngents, arrivalRateMin, arrivalRateMax, custm, threshold))

In [8]:
print_statistics(devs)


Average waiting times: 5.31 0.08 0.20 0.29

Probabilities that a customer has to wait: 0.60 0.03 0.10 0.14

Probabilities of an Idle server: 0.81 0.73 0.70 0.77

Average service times: 7.24 7.07 7.38 6.85

Average times between arrivals: 2.77 2.46 2.47 2.47

Average waiting times for those who wait: 8.85 2.67 2.00 2.07

Average times a customer spends in the system: 12.55 7.15 7.58 7.14


In [11]:
servers_num = 6
devs = []
for task in range(1, 5):
    devs.append(run_devs(task, servers_num, maxAngents, arrivalRateMin, arrivalRateMax, custm, threshold))

In [12]:
print_statistics(devs)


Average waiting times: 1.83 0.01 0.00 0.00

Probabilities that a customer has to wait: 0.34 0.01 0.00 0.00

Probabilities of an Idle server: 0.98 0.98 0.98 0.99

Average service times: 7.05 7.39 7.20 7.18

Average times between arrivals: 2.48 2.48 2.35 2.63

Average waiting times for those who wait: 5.38 1.00 0.00 0.00

Average times a customer spends in the system: 8.88 7.40 7.20 7.18


In [16]:
# Видно, что 4-ая система наиболее эффективная, первая - наименее.
# При увеличении количества серверов, при заданных параметрах, для задач 2,3,4 система выполняет запросы практически без 
# необходимости клиенту ждать в очереди